# client instance 생성

In [1]:
DEPLOYMENT_URL="http://10.8.1.229:8000/"
DEPLOYMENT_URL="http://localhost:2024/"

from langgraph_sdk import get_client

client = get_client(url=DEPLOYMENT_URL)

# thread 생성 (앨런의 channel에 대응)

- 생성 요청 후 thread_id를 포함한 데이터를 return 받음
- 하나의 thread에 대해서 여러 에이전트를 호출하고 대화가능
- 현재 앨런은 한개의 channel에 대해서 한개의 agent가 1:1로 대응되므로 현재는 고려할 필요 없지만, 추후에 1개의 채널에 대해서 여러 agent를 호출가능

In [2]:
thread = await client.threads.create()
print(thread)

thread_id = thread["thread_id"]
print(thread_id)

{'thread_id': '52a31f21-57db-4cff-b829-6568c3a9e798', 'created_at': '2025-08-12T01:48:05.888427+00:00', 'updated_at': '2025-08-12T01:48:05.888430+00:00', 'metadata': {}, 'status': 'idle', 'config': {}, 'values': None}
52a31f21-57db-4cff-b829-6568c3a9e798


# 에이전트 호출

- 해당 thread에서 어떤 agent를 호출할 지 assistant_id를 포함하여 요청을 보냄

## parameters
- thread_id : 위에서 생성한 thread_id
- assistant_id : 앨런의 persona_id에 대응, agent를 구분하는 id, 슬라이드 에이전트의 경우 "slide_generate_agent"
- input : 해당 agent 요청에 대한 parameter, 현재는 messages만 존재, 앨런 페이지에서 유저로 부터 받는 input을 해당 parameter에 dict로 넣어줌
- stream_mode : 어떤 event에 따라서 메시지를 받을지 지정, updates(한 단계가 끝날 때마나 메시지가 옴), events(llm이 생성하는 단위로 메시지 생성)

In [29]:
def filter_ai_msg(chunk):
    # 유저와 채팅 창에 출력될 ai 메시지를 필터링하는 함수
    if 'updates' in chunk.event:
        node_name = chunk.data.keys()
        if 'agent' in node_name:
            if chunk.data['agent']['messages'][-1]["name"] != None:
                return True
    return False

def filter_silde_data(chunk):
    # slide 관련 데이터를 담은 메시지를 필터링하는 함수
    # 슬라이드 한페이지 단위로 메시지가 옴
    if 'updates' in chunk.event:
        node_name = chunk.data.keys()
        if 'executor' in node_name:
            return True
            print(chunk.data['agent']['messages'][-1]['content'])

In [4]:
assistant_id = "slide_generate_agent"
list_chunk = list()

In [5]:

input = { "messages": "안녕"}

async for chunk in client.runs.stream(
    thread_id,
    assistant_id=assistant_id,
    input=input,
    stream_mode=["updates"],
    stream_subgraphs=True
):
    try:
        list_chunk.append(chunk)
        if filter_ai_msg(chunk):
            print("#" * 10 + "  AI 메시지  " + "#" * 10)
            print(chunk.data['agent']['messages'][-1]['content'])
        elif filter_silde_data(chunk):
            print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
            print(chunk)
    except:
        continue
    pass

##########  AI 메시지  ##########
안녕하세요! 슬라이드를 생성하는 데 필요한 정보를 도와드릴 수 있습니다. 어떤 주제로 슬라이드를 만들고 싶으신가요?


In [6]:
input = { "messages": "제주도 여행을 주제로 슬라이드를 만들어줘"}

async for chunk in client.runs.stream(
    thread_id,
    assistant_id=assistant_id,
    input=input,
    stream_mode=["updates"],
    stream_subgraphs=True
):
    try:
        list_chunk.append(chunk)
        if filter_ai_msg(chunk):
            print("#" * 10 + "  AI 메시지  " + "#" * 10)
            print(chunk.data['agent']['messages'][-1]['content'])
        elif filter_silde_data(chunk):
            print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
            print(chunk)
    except:
        continue
    pass

##########  AI 메시지  ##########
좋아요, 제주도 여행을 주제로 슬라이드를 만들어 보겠습니다. 슬라이드를 만드는 데 도움이 될 몇 가지 질문을 드릴게요.

1. 슬라이드의 주요 내용을 무엇으로 하고 싶으신가요? 예를 들어, 관광지, 음식, 문화 등.
2. 슬라이드의 분량은 어느 정도로 생각하고 계신가요?
3. 특정 색상 테마나 스타일을 원하시나요?
4. 추가하고 싶은 이미지나 그래픽 요소가 있나요?
5. 발표 목적이나 대상이 어떻게 되나요? 예를 들어, 친구들과의 여행 계획, 여행 가이드 작성 등.

이 질문들에 답변해주시면 더욱 맞춤형 슬라이드를 만들어드릴 수 있을 것 같아요.


In [14]:
input = {"messages": "알아서 해줘"}

async for chunk in client.runs.stream(
    thread_id,
    assistant_id=assistant_id,
    input=input,
    stream_mode=["updates"],
    stream_subgraphs=True
):
    try:
        list_chunk.append(chunk)
        if filter_ai_msg(chunk):
            print("#" * 10 + "  AI 메시지  " + "#" * 10)
            print(chunk.data['agent']['messages'][-1]['content'])
            print(chunk.data)
        elif filter_silde_data(chunk):
            print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
            print(chunk)
    except:
        continue
    pass

##########  AI 메시지  ##########

{'agent': {'messages': [{'content': '', 'additional_kwargs': {'tool_calls': [{'id': 'call_6OTOnJTLDttuqvHVSxRqJsvo', 'function': {'arguments': '{}', 'name': 'transfer_to_slide_create_agent'}, 'type': 'function'}], 'refusal': None}, 'response_metadata': {'token_usage': {'completion_tokens': 14, 'prompt_tokens': 793, 'total_tokens': 807, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_5d58a6052a', 'id': 'chatcmpl-C3YS5bktuUi8aGjwFZWoCsddmlhjB', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None, 'content_filter_results': {}}, 'type': 'ai', 'name': 'supervisor', 'id': 'run--b6b025df-c44a-4501-b465-38634509a3cd-0', 'example': False, 'tool_calls': [{'name': 'transfer_to_slide_create_agent', 'args': {}, 'id': 'call_6OTOnJTLDtt

In [11]:
for chunk in list_chunk:
    if filter_silde_data(chunk):
        print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
        print(chunk.data)
        break

##########  슬라이드 데이터  ##########
{'executor': {'slides': [{'slide_type': 'title', 'topic': '제주도 여행', 'idx': 0, 'name': '타이틀 슬라이드', 'description': '제주도 여행에 대한 프레젠테이션 타이틀 페이지', 'requirements': ['타이틀 텍스트', '부제목 또는 설명'], 'research': True, 'content': '## 타이틀 슬라이드: 제주도 여행 프레젠테이션\n\n제주도는 남한의 남쪽 해안에 위치한 아름다운 섬으로, 그 독특한 화산 지형과 풍부한 자연경관으로 세계적으로 유명합니다. 주요 명소에는 한라산, 용암 지형, 그리고 특별히 지정된 유산지들이 포함되어 있어, 관광객들에게 다양한 볼거리와 경험을 제공합니다. \n\n또한, 제주는 한국에서도 최고의 관광지 중 하나로 손꼽히며, 연중 내내 다양한 활동과 명소를 즐길 수 있는 곳입니다. 자연을 사랑하는 사람에게는 하이킹과 해변 활동이 준비되어 있으며, 문화에 관심이 있는 여행자들에게는 독특한 현지 문화와 요리를 경험할 기회를 제공합니다 [1][2].\n\n제주도 여행은 아름다운 해안선, 고대의 숲, 그리고 현대화의 조화를 직접 체험할 수 있는 특별한 기회를 제공합니다. 이러한 매력을 갖춘 제주도는 모든 여행자에게 잊지 못할 추억을 선사할 것입니다 [3][4].\n\n### Sources\n[1] Quick Guide to Jeju - There She Goes Again: https://thereshegoesagain.org/quick-guide-to-jeju/\n[2] Jeju Island, a Picturesque Paradise for a Peaceful Retreat - Visit Korea: https://english.visitkorea.or.kr/svc/contents/contentsView.do?vcontsId=222085\n[3] Why Is Jeju Island, So

In [35]:
for chunk in list_chunk:
    if filter_ai_msg(chunk):
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        print(chunk.data['agent']['messages'][-1]['content'])

##########  AI 메시지  ##########
안녕하세요! 슬라이드를 생성하는 데 필요한 정보를 도와드릴 수 있습니다. 어떤 주제로 슬라이드를 만들고 싶으신가요?
##########  AI 메시지  ##########
좋아요, 제주도 여행을 주제로 슬라이드를 만들어 보겠습니다. 슬라이드를 만드는 데 도움이 될 몇 가지 질문을 드릴게요.

1. 슬라이드의 주요 내용을 무엇으로 하고 싶으신가요? 예를 들어, 관광지, 음식, 문화 등.
2. 슬라이드의 분량은 어느 정도로 생각하고 계신가요?
3. 특정 색상 테마나 스타일을 원하시나요?
4. 추가하고 싶은 이미지나 그래픽 요소가 있나요?
5. 발표 목적이나 대상이 어떻게 되나요? 예를 들어, 친구들과의 여행 계획, 여행 가이드 작성 등.

이 질문들에 답변해주시면 더욱 맞춤형 슬라이드를 만들어드릴 수 있을 것 같아요.
##########  AI 메시지  ##########

##########  AI 메시지  ##########
제주도 여행을 주제로 한 슬라이드 준비를 완료했습니다. 다음과 같은 내용을 포함하도록 하겠습니다:

1. 제주도의 주요 관광 명소 소개: 성산 일출봉, 한라산 국립공원 등을 포함한 인기 명소
2. 숨겨진 명소: 로컬 추천지 등 덜 알려진 특별한 장소
3. 제주도의 전통 음식: 갈치국, 고기국수 등
4. 제주도의 역사와 문화: 탐라국부터 현대까지의 역사적 배경

이제 이 정보를 기반으로 슬라이드를 생성하겠습니다.
##########  AI 메시지  ##########

##########  AI 메시지  ##########
제주도 여행에 대한 슬라이드를 만들기 위해 준비된 내용은 다음과 같습니다:

1. 제주도의 주요 관광 명소 소개: 성산 일출봉, 한라산 국립공원 등을 포함한 인기 명소
2. 숨겨진 명소: 로컬 추천지 등 덜 알려진 특별한 장소
3. 제주도의 전통 음식: 갈치국, 고기국수 등
4. 제주도의 역사와 문화: 탐라국부터 현대까지의 역사적 배경

슬라이드 생성을 위해 충분한 

In [37]:
state = await client.threads.get_state(thread_id=thread_id)

for msg in state["values"]["messages"]:
    if msg["type"] != "human":
        print("AI :", msg["content"])
        print(msg['name'])

AI : 안녕하세요! 슬라이드를 생성하는 데 필요한 정보를 도와드릴 수 있습니다. 어떤 주제로 슬라이드를 만들고 싶으신가요?
supervisor
AI : 좋아요, 제주도 여행을 주제로 슬라이드를 만들어 보겠습니다. 슬라이드를 만드는 데 도움이 될 몇 가지 질문을 드릴게요.

1. 슬라이드의 주요 내용을 무엇으로 하고 싶으신가요? 예를 들어, 관광지, 음식, 문화 등.
2. 슬라이드의 분량은 어느 정도로 생각하고 계신가요?
3. 특정 색상 테마나 스타일을 원하시나요?
4. 추가하고 싶은 이미지나 그래픽 요소가 있나요?
5. 발표 목적이나 대상이 어떻게 되나요? 예를 들어, 친구들과의 여행 계획, 여행 가이드 작성 등.

이 질문들에 답변해주시면 더욱 맞춤형 슬라이드를 만들어드릴 수 있을 것 같아요.
supervisor
AI : 
supervisor
AI : Successfully transferred to slide_create_agent
transfer_to_slide_create_agent
AI : 검색 도구를 사용하여 목차를 생성하기 위한 조사를 시작합니다.
None
AI : 생성된 목차는 다음과 같습니다. 

2. 제주도 여행 - 제주도에서 방문할 수 있는 관광 명소를 소개합니다. 성산 일출봉, 한라산 국립공원과 같은 유명 명소부터 숨겨진 보석 같은 장소까지 다양한 볼거리를 제공합니다. 
3. 제주도 여행 - 제주도에서 잘 알려지지 않은 숨은 명소들을 소개합니다. 로컬들이 추천하는 독특하고 매력적인 장소를 만나보세요. 
4. 제주도 여행 - 제주도에서만 맛볼 수 있는 전통 음식을 소개합니다. 갈치국, 고기국수와 같은 향토 음식을 통해 제주의 맛을 경험해보세요. 
5. 제주도 여행 - 제주도의 깊은 역사와 독특한 문화를 탐구합니다. 탐라국 시절부터 현대에 이르기까지 제주도의 역사와 문화의 변천사를 알아봅니다. 

None
AI : Transferring back to supervisor
slide_create_agent
AI : Successfully tr

In [ ]:
state["values"]["slides"]